In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U sentence-transformers > /dev/null

In [ ]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 5.5 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!ls 'drive/MyDrive/Colab Notebooks/nlp/clustering kpa'

'11_legal_pegasus_rephrase results.ipynb'
 1_KPA_clustering.ipynb
 2_KPA_clustering_fine-tunning.ipynb
 3_legal_clustering.ipynb
 4_legal_barh_clustering.ipynb
 5_legal_clustering_legal_bert.ipynb
 6_legal_clustering_topic_bert.ipynb
 7_explore_final.ipynb
 8_pegasus_sum_berttopic.ipynb
 9_gensim_sum_berttopic.ipynb
 abstractive_summarization_bertsumabs_cnndm.ipynb
 abstractive_summarization_minilm_cnndm.ipynb
'Copy of 10_legal_pegasus_sum_berttopic.ipynb'
 data


In [ ]:
import json
import sklearn
import sklearn.model_selection
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

In [ ]:
random_state = 321

# Prepare data

# Pegasus

In [ ]:
!pip install transformers -q > /dev/null
!pip install sentencepiece -q > /dev/null
!pip install rouge_score -q > /dev/null

In [ ]:
import nltk
import torch
from collections import OrderedDict
from transformers import set_seed, PegasusTokenizer, PegasusForConditionalGeneration

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

In [ ]:
print(device)
model.to(device)
print('')

cuda



In [ ]:
def process_arg(arg):
  arg = arg.replace('\n',' ')
  arg = arg.replace('\r','')
  arg = arg.replace('\"','')
  arg = arg.replace('\'','')
  arg = arg.replace('  ',' ')
  arg = arg.strip(' .,')
  # arg = arg.strip('.')
  return arg

def summarize_func(text):
  input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
  params = {
  'num_beams':9,
  'no_repeat_ngram_size':3,
  'length_penalty':35.0,
  'min_length':1,
  'max_length':input_tokenized.shape[1], #20,
  'early_stopping':True
  }
  summary_ids = model.generate(input_tokenized, **params)
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
  return summary

def run_summarization_kp(df, verbose=False):
  df['kp_rephrased'] = 'no'
  for i,group in df.groupby(['name','cluster_id']):
    kp = group['kp'].tolist()[0]
    args = [process_arg(kp)]
    args += [process_arg(arg) for arg in group['argument'].tolist()]
    ids = group['id'].tolist()
    text = '. '.join(args)
    sum = summarize_func(text)
    if verbose:
      print(i)
      print('IDS')
      print(ids)
      print('KP')
      print(kp)
      print('ARGS')
      print(args)
      print('TEXT')
      print(text)
      print('SUM')
      print(sum)
    for id in ids:
      df.loc[df['id']==id, 'kp_rephrased']=str(sum) 

In [ ]:
df = pd.read_excel('drive/MyDrive/Colab Notebooks/nlp/clustering kpa/data/echr_arguments_kp_berttopic_lex_rank.xls')
df.head(2)

,id,name,cluster_id,argument,kp
0,1,00.txt,0,The Commission notes that the applicant was de...,The Commission notes that the applicant was de...
1,2,00.txt,0,He was released after the Court of Appeal revi...,The Commission notes that the applicant was de...


In [ ]:
%%time
df_new = run_summarization_kp(df,verbose=True)

('00.txt', -1)
IDS
[7, 8, 15, 16, 18, 19, 20, 22, 23, 25, 26, 28, 35, 36, 39, 46]
KP
In the present case, the Commission notes that none of these complaints has been raised either formally or even in substance during the proceedings before the Court of Cassation.
ARGS
['In the present case, the Commission notes that none of these complaints has been raised either formally or even in substance during the proceedings before the Court of Cassation', 'The applicant was not detained on remand prior to his trial and the judicial control of the lawfulness of his subsequent detention after conviction was provided by the first instance court (cf. Eur. Court H.R., De Wilde, Ooms and Versyp judgment of 18 June 1971, Series A no. 12, p. 40, para. 76)', 'However, again, the Commission finds no evidence in the case to substantiate this complaint', 'For this condition to be fulfilled, it is not sufficient for the applicant merely to have presented his case to the various courts competent to deal with

KeyboardInterrupt: ignored

In [ ]:
df['kp_rephrased'][13]

'The Commission finds that the applicant was deprived of his liberty after conviction by a competent court within the meaning of Article 5 para. 1 (a) of the Convention. This right shall include freedom to hold opinions and to receive and impart information and ideas without interference by public authorities and regardless of frontiers. The Commission observes that Article 26 should be applied with some degree of flexibility and without excessive formalism. Article 3 of Protocol No. 1 provides for free elections under conditions which will ensure the free expression of the opinion of the people in the choice of the legislature. No other ground for declaring this part of the case inadmissible has been established. It is sufficient that the complaints intended to be raised subsequently before the Convention organs should have been raised at least in substance and in compliance with the formal requirements and time-limits laid down in domestic law.'

# Lex Rephrased

In [ ]:
df = pd.read_excel('drive/MyDrive/Colab Notebooks/nlp/clustering kpa/data/echr_arguments_kp_berttopic_lex_rank.xls')
df.head(2)

,id,name,cluster_id,argument,kp
0,1,00.txt,0,The Commission notes that the applicant was de...,The Commission notes that the applicant was de...
1,2,00.txt,0,He was released after the Court of Appeal revi...,The Commission notes that the applicant was de...


In [ ]:
df_lex_rephrased = df.copy()